In [1]:
from pyarts.workspace import Workspace
ws = Workspace()
ws.stokes_dim

OSError: [Errno The following error was encountered when trying to load the ARTS API: ] dlopen(/Users/olemke/Hacking/git/arts/build/python/pyarts/workspace/libarts_api.so, 6): image not found